In [59]:
%%time
#import findspark
#findspark.init()
from pyspark import SparkContext,SparkConf

sc.stop()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 634 ms


In [49]:
def create_sc(pyFiles):
    sc_conf = SparkConf()
    sc_conf.setAppName("Weather_PCA")
    sc_conf.set('spark.executor.memory', '2g')
    sc_conf.set('spark.executor.cores', '4')
    sc_conf.set('spark.cores.max', '40')
    sc_conf.set('spark.default.parallelism','120')
    sc_conf.set('spark.logConf', True)
    print(sc_conf.getAll())

    sc = SparkContext(conf=sc_conf,pyFiles=pyFiles)

    return sc 

sc = create_sc(pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStatistics.py'])

[('spark.eventLog.enabled', 'true'), ('spark.driver.extraLibraryPath', '/usr/lib/hadoop/lib/native:/usr/lib/hadoop-lzo/lib/native'), ('spark.blacklist.decommissioning.timeout', '1h'), ('spark.app.name', 'Weather_PCA'), ('spark.executorEnv.PYSPARK_PYTHON', '/usr/bin/python3'), ('spark.yarn.appMasterEnv.SPARK_PUBLIC_DNS', '$(hostname -f)'), ('spark.executor.cores', '4'), ('spark.cores.max', '40'), ('spark.executor.extraJavaOptions', "-verbose:gc -XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=70 -XX:MaxHeapFreeRatio=70 -XX:+CMSClassUnloadingEnabled -XX:OnOutOfMemoryError='kill -9 %p'"), ('spark.ui.proxyBase', '/proxy/application_1522778993540_0003'), ('spark.eventLog.dir', 'hdfs:///var/log/spark/apps'), ('spark.sql.hive.metastore.sharedPrefixes', 'com.amazonaws.services.dynamodbv2'), ('spark.sql.warehouse.dir', 'hdfs:///user/spark/warehouse'), ('spark.submit.deployMode', 'client'), ('spark.history.fs.logDirectory', 'hdfs:///var/log/sp

In [50]:
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [35]:
import numpy as np
from lib.computeStatistics import *

In [36]:
!aws s3 cp --recursive --quiet  s3://dse-weather/NY.parquet /mnt/workspace/Data/NY.parquet

In [37]:
!du /mnt/workspace/Data/

424	/mnt/workspace/Data/US_stations.parquet/US_stations.parquet
848	/mnt/workspace/Data/US_stations.parquet
990536	/mnt/workspace/Data/US_weather.parquet/US_weather.parquet
1981072	/mnt/workspace/Data/US_weather.parquet
28244	/mnt/workspace/Data/NY.parquet
2010164	/mnt/workspace/Data/


In [38]:
!hdfs dfs -mkdir weather
!hdfs dfs -ls

mkdir: `weather': File exists
Found 2 items
drwxr-xr-x   - hadoop hadoop          0 2018-04-03 20:00 .sparkStaging
drwxr-xr-x   - hadoop hadoop          0 2018-04-03 19:44 weather


In [39]:
!hdfs dfs -copyFromLocal /mnt/workspace/Data/NY.parquet /weather/NY.parquet

copyFromLocal: `/weather/NY.parquet/NY.parquet/_SUCCESS': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00000-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00001-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00002-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00003-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00004-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00005-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY.parquet/NY.parquet/part-00006-8342bcf4-7fc2-4183-8e11-aefdb4915fbb-c000.snappy.parquet': File exists
copyFromLocal: `/weather/NY

In [40]:
!hdfs dfs -du /weather/

57771504    /weather/NY.parquet
847798      /weather/US_stations.parquet
2027712578  /weather/US_weather.parquet


In [51]:
%%time
df=sqlContext.read.parquet('/weather/NY.parquet')
print(df.count())
df.show(5)

84199
+-----------+-----------+----+--------------------+-----+
|    Station|Measurement|Year|              Values|State|
+-----------+-----------+----+--------------------+-----+
|USC00303452|       PRCP|1903|[00 7E 00 7E 00 7...|   NY|
|USC00303452|       PRCP|1904|[00 00 28 5B 00 0...|   NY|
|USC00303452|       PRCP|1905|[00 00 60 56 60 5...|   NY|
|USC00303452|       PRCP|1906|[00 00 00 00 00 0...|   NY|
|USC00303452|       PRCP|1907|[00 00 00 00 60 5...|   NY|
+-----------+-----------+----+--------------------+-----+
only showing top 5 rows

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 11.3 s


In [52]:
from time import time
t=time()

N=sc.defaultParallelism
print('Number of executors=',N)
print('took',time()-t,'seconds')

Number of executors= 120
took 0.0011816024780273438 seconds


In [53]:
data_dir='/mnt/workspace/Data/'


In [54]:
%%time
from pickle import dump

state='NY'
parquet=state+'.parquet'
df=sqlContext.read.parquet('/weather/'+parquet).cache()
print(state,df.count())

NY 84199
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 6.69 s


In [55]:
%%time
df.show(4)

+-----------+-----------+----+--------------------+-----+
|    Station|Measurement|Year|              Values|State|
+-----------+-----------+----+--------------------+-----+
|USC00303452|       PRCP|1903|[00 7E 00 7E 00 7...|   NY|
|USC00303452|       PRCP|1904|[00 00 28 5B 00 0...|   NY|
|USC00303452|       PRCP|1905|[00 00 60 56 60 5...|   NY|
|USC00303452|       PRCP|1906|[00 00 00 00 00 0...|   NY|
+-----------+-----------+----+--------------------+-----+
only showing top 4 rows

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 74.7 ms


In [56]:
import os
os.environ['PYSPARK_PYTHON'], os.environ['PYSPARK_DRIVER_PYTHON']

('/usr/bin/python3', '/usr/bin/python3')

In [57]:
Environ_vars=['SPARK_HOME','PYSPARK_PYTHON', 'PYSPARK_DRIVER_PYTHON']
for var in Environ_vars:
    print(var,var in os.environ)


SPARK_HOME True
PYSPARK_PYTHON True
PYSPARK_DRIVER_PYTHON True


### Compatibility error

The relevant part of the error messages is:
```
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.
```

If I try to check the variables using the terminal:
```sh
echo $PYSPARK_PYTHON, $PYSPARK_DRIVER_PYTHON
```

I get:
```
,
```

In [58]:
%%time
STAT=computeStatistics(sqlContext,df)

TMAX : shape of mdf is  13437
time for TMAX is 20.44739818572998
SNOW : shape of mdf is  15629
time for SNOW is 19.603999137878418
SNWD : shape of mdf is  14617
time for SNWD is 18.709880113601685
TMIN : shape of mdf is  13442
time for TMIN is 16.583598852157593
PRCP : shape of mdf is  16118
time for PRCP is 20.742507696151733
TOBS : shape of mdf is  10956
time for TOBS is 14.593730926513672
CPU times: user 4.47 s, sys: 292 ms, total: 4.76 s
Wall time: 1min 50s


### Run time with 10 executors, 2GB each
```
TMAX : shape of mdf is  13437
time for TMAX is 32.62236022949219
SNOW : shape of mdf is  15629
time for SNOW is 30.68799138069153
SNWD : shape of mdf is  14617
time for SNWD is 29.275496244430542
TMIN : shape of mdf is  13442
time for TMIN is 27.989774703979492
PRCP : shape of mdf is  16118
time for PRCP is 32.24087643623352
TOBS : shape of mdf is  10956
time for TOBS is 24.352991819381714
CPU times: user 4.36 s, sys: 252 ms, total: 4.61 s
Wall time: 2min 57s
```
### Run time with 40 executors, 16GB each.

```
TMAX : shape of mdf is  13437
time for TMAX is 19.664730310440063
SNOW : shape of mdf is  15629
time for SNOW is 18.108243942260742
SNWD : shape of mdf is  14617
time for SNWD is 16.28211808204651
TMIN : shape of mdf is  13442
time for TMIN is 16.398584127426147
PRCP : shape of mdf is  16118
time for PRCP is 19.20434069633484
TOBS : shape of mdf is  10956
time for TOBS is 14.464087009429932
CPU times: user 4.54 s, sys: 292 ms, total: 4.83 s
Wall time: 1min 44s
```
### Run time with 120 executorsm 2GB each
```
TMAX : shape of mdf is  13437
time for TMAX is 20.44739818572998
SNOW : shape of mdf is  15629
time for SNOW is 19.603999137878418
SNWD : shape of mdf is  14617
time for SNWD is 18.709880113601685
TMIN : shape of mdf is  13442
time for TMIN is 16.583598852157593
PRCP : shape of mdf is  16118
time for PRCP is 20.742507696151733
TOBS : shape of mdf is  10956
time for TOBS is 14.593730926513672
CPU times: user 4.47 s, sys: 292 ms, total: 4.76 s
Wall time: 1min 50s
```

In [28]:
filename=data_dir+'STAT_%s.pickle'%state
!ls -l $data_dir
dump((STAT,STAT_Descriptions),open(filename,'wb'))

total 32
drwxrwxr-x 2 hadoop hadoop  4096 Apr  1 17:49 NY.parquet
drwxrwxr-x 3 hadoop hadoop  4096 Apr  1 16:23 US_stations.parquet
drwxrwxr-x 3 hadoop hadoop 20480 Apr  1 16:23 US_weather.parquet


NameError: name 'STAT' is not defined

In [9]:
X=STAT['TMAX']['Var']
for key in STAT.keys():
    Y=STAT[key]['Var']
    print(key,sum(abs(X-Y)))

TMAX 0.0
SNOW 852107.705867
SNWD 4464167.85212
TMIN 319734.5315
PRCP 1184305.12284
TOBS 277719.008938


In [11]:
!ls -l ../../Data/Weather/STAT*

-rw-r--r--  1 yoavfreund  staff  25684434 Mar 18 20:30 ../../Data/Weather/STAT_NY.pickle
-rw-r--r--  1 yoavfreund  staff  17545930 Mar 18 17:38 ../../Data/Weather/STAT_NY.pickle.gz
-rw-r--r--  1 yoavfreund  staff  26741490 Mar 18 20:19 ../../Data/Weather/STAT_RI.pickle
-rw-r--r--  1 yoavfreund  staff  13522496 Mar 10 12:06 ../../Data/Weather/STAT_SSSSBBBB.pickle.gz


In [12]:
!gzip -f ../../Data/Weather/STAT*.pickle
!ls -l ../../Data/Weather/STAT*

-rw-r--r--  1 yoavfreund  staff  14948048 Mar 18 20:30 ../../Data/Weather/STAT_NY.pickle.gz
-rw-r--r--  1 yoavfreund  staff  11717259 Mar 18 20:19 ../../Data/Weather/STAT_RI.pickle.gz
-rw-r--r--  1 yoavfreund  staff  13522496 Mar 10 12:06 ../../Data/Weather/STAT_SSSSBBBB.pickle.gz


In [13]:
for state in ['RI','NY']:
    command="aws s3  cp ../../Data/Weather/STAT_%s.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_%s.pickle.gz"%(state,state)
    print(command)
    !$command

aws s3  cp ../../Data/Weather/STAT_RI.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_RI.pickle.gz
upload: ../../Data/Weather/STAT_RI.pickle.gz to s3://mas-dse-open/Weather/by_state/STAT_RI.pickle.gz
aws s3  cp ../../Data/Weather/STAT_NY.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_NY.pickle.gz
upload: ../../Data/Weather/STAT_NY.pickle.gz to s3://mas-dse-open/Weather/by_state/STAT_NY.pickle.gz


In [14]:
!aws s3  ls s3://mas-dse-open/Weather/by_state/ | grep STAT

2018-03-18 20:34:12   14948048 STAT_NY.pickle.gz
2018-03-18 20:33:54   11717259 STAT_RI.pickle.gz
